In [2]:
from spectacle import *

Manim Community v0.14.0

Welcome to spectacle


In [76]:
%%manim -v WARNING -ql FeynmanVideo

def make_electron_ball():
    return VGroup(Circle(radius=0.25,color=GREEN,fill_opacity=1), MathTex(ELECTRON, color=BLACK))

def make_electron_shell():
    return VGroup(Circle(radius=0.25,color=GREEN), MathTex(ELECTRON))

class ElectricCharge(VGroup):
    def __init__(self, center=ORIGIN, init_velocity=ORIGIN, charge=1, is_shell=False):
        self.center = center
        self.charge = charge
        self.is_shell = is_shell
        mobject_func = {True: make_electron_shell}.get(is_shell, make_electron_ball) 
        
        self.mobject = mobject_func().move_to(center)

        self.velocity = init_velocity
        
        self.acceleration = point(0,0)
        # self.acceleration_vector = Arrow(start=self.center, end=self.center+self.acceleration, buff=0)
        
        vmobjects = [self.mobject]
        
    
        
        
        super().__init__(*vmobjects)
    
    def get_center(self):
        return self.center    
    
    def get_source_func(self, p):
        x, y, = p[0], p[1]
        x0, y0 = self.center[0], self.center[1]
        r = np.linalg.norm(point(x-x0, y-y0))
        if r <= 0.15:
            return point(0, 0)
        efield_mag = 1/(r**3)
        return self.charge*point((x-x0)*efield_mag,(y-y0)*efield_mag)
    
    def get_ball(self):
        return self.mobject
    
    
    def update_position_by_field(self, mobject, dt):
        self.acceleration = self.charge * self.field_func(self.center)
        self.velocity = self.velocity + self.acceleration*dt
        self.center = self.center + self.velocity * dt
        
        mobject.move_to(self.center)
        
    def get_updater_by_field(self, field_func):
        self.field_func = field_func
        return lambda mob, dt: self.update_position_by_field(mob, dt)
        

class FeynmanVideo(Scene):
    
    def construct(self):
        self.scene1()
        self.scene2()
        
    def scene1(self):
        """Scene 1:
        Show an electron generating an electric field
        """
        bound = 10
        dx = 0.5
        x_diff = 2
        
        held_electron = ElectricCharge(center=LEFT*x_diff, charge=10)
        efield = ArrowVectorField(held_electron.get_source_func)
        self.add(efield, held_electron.get_ball().shift(OUT))
        
        # efield.start_animation(warm_up=False, flow_speed=1.5)
        
        self.wait(4)
        
        
        free_electron = ElectricCharge(center=RIGHT*x_diff+DOWN*4.2, init_velocity=point(-0.85,2), is_shell=True)
        self.add(free_electron)
        
        free_electron.add_updater(free_electron.get_updater_by_field(held_electron.get_source_func))
        
        self.play(FadeIn(free_electron),run_time=0.2)
        
        self.wait(10)
        
        self.play(efield.animate.set_color(BLACK), FadeOut(held_electron), FadeOut(free_electron), run_time=2)
        
    def scene2(self):
        """Scene 2:
        Show two electrons repelling
        """
        x_diff = 2
        
        field_height = 0.25
        
        left_center = LEFT*x_diff+DOWN*4.2
        right_center = RIGHT*x_diff+DOWN*4.2
        
        free_electron_left = ElectricCharge(center=left_center, init_velocity=point(0.75,1.8), is_shell=True, charge=2)
        free_electron_right = ElectricCharge(center=right_center, init_velocity=point(-0.75,1.8), is_shell=True, charge=2)
        
        x_range = [left_center[0], right_center[0], 0.5]
        y_range = [left_center[1]-field_height,left_center[1]+ field_height, field_height/5]
        
        efield_left = ArrowVectorField(free_electron_left.get_source_func)
        efield_right = ArrowVectorField(free_electron_right.get_source_func)
        
        self.add(free_electron_left, free_electron_right, efield_left, efield_right)
        
        free_electron_left.add_updater(free_electron_left.get_updater_by_field(free_electron_right.get_source_func))
        free_electron_right.add_updater(free_electron_right.get_updater_by_field(free_electron_left.get_source_func))
        
        
        
        efield_left.add_updater(lambda field: field.become(
            ArrowVectorField(
                lambda p: free_electron_left.get_source_func(p),
            )
        ))
        
        efield_right.add_updater(lambda field: field.become(
            ArrowVectorField(
                lambda p: free_electron_right.get_source_func(p),
            )
        ))
        
        self.wait(1)
        
        
        
        
        
        
        
        
        
        # efield.start_animation(flow_speed=3)
        # self.wait(3*efield.virtual_time/efield.flow_speed)
        
        